# Crawling Web Berita CNN Indonesia


## Apa itu Crawling?


Crawling merupakan proses search engine untuk menemukan konten atau sesuatu situs halaman yang ada. Dalam bahasa kerennya crawling atau web crawling merupakan proses dimana search engine mengirimkan bot atau robot yang disebut (crawler atau spider) yang digunakan untuk menemukan konten-konten yang ada.

Yang dimaksud konten yaitu bervariasi, mulai dari halaman website yang saya lakukan ini, kemudian gambar, video, dokumen, dan lain sebagainya. Seperti halnya laba-laba, datang ke sebuah jaring dan melihat beberapa halaman website, kemudian mengikuti link yang terdapat di halaman website tersebut untuk mencari URL yang baru.

Ketika ada pengguna yang mencari sebuah konten di search engine dengan keyword tertentu, search engine akan mencarinya di indeks dan menentukan konten mana yang paling sesuai untuk pengguna tersebut.


## Proses Crawling


### Tool atau Library


In [9]:
# Import Library yang dibutuhkan
from bs4 import BeautifulSoup as bs
import pandas as pd
from tqdm import tqdm

- **Request** digunakan untuk mengambil html/http dari sebuah website.
- **BeautifulSoup** berfungsi untuk mengambil data dari html/xml.
- **Time** berfungsi untuk memberikan jeda ketika ingin berpindah halaman.
- **Pandas** digunakan untuk membuat dataframe agar mudah dibaca.
- **tqdm** Untuk mentracking proses program


### Code Program


#### Fungsi Clean_text()


In [10]:
def clean_text(text):
	return text.strip()

Fungsi untuk membersihkan text yang tidak diinginkan, atau mengganggu.


#### Fungsi scrape_news()


In [11]:
def scrape_news(soup):
	berita = {}
	texts = []
	# TODO:
	# ada struktur aneh https://www.cnnindonesia.com/olahraga/20240830134615-142-1139388/live-report-timnas-indonesia-vs-thailand-u-20
	
	berita["judul"] = soup.title.text

	if 'FOTO:' in berita["judul"]:
		div_content = soup.find("div", class_="detail-text text-cnn_black text-sm grow min-w-0")
		if div_content:
			full_text = div_content.get_text(strip=True)
			text = full_text.split('--', 1)[-1]
			text = text.split('var article')[0].strip()

			cleaned_text = clean_text(text)
			texts.append(cleaned_text)

		berita["tanggal"] = soup.find("div", class_="container !w-[1100px] overscroll-none").find("div", class_="text-cnn_grey text-sm").text

	else:
		text_list = soup.find("div", class_="detail-text text-cnn_black text-sm grow min-w-0")
		for text in text_list.find_all("p"):
			if 'para_caption' not in text.get('class', []):
				cleaned_text = clean_text(text.text)
				texts.append(cleaned_text)

		berita["tanggal"] = soup.find("div", class_="container !w-[1100px] overscroll-none").find("div", class_="text-cnn_grey text-sm mb-4").text

	berita["isi"] = "\n".join(texts)
	berita["kategori"] = soup.find("meta", attrs={'name': 'dtk:namakanal'})['content']
	return berita

Pada fungsi ini berisikan proses pembedahan dan juga pengambilan data pada sebuah website. Mengambil data sesuai struktur HTML/web yang ingin diambil datanya.


In [12]:
def hide_ads(driver):
	try:
		js_code = """
				var ads = document.querySelectorAll(
						'.ad-container, .banner-ad, [data-ad-unit], .ads-slot, [data-google-query-id], .bottom_banner_bar, .bn_skin, imp-css, mgbox, ad_unit, videoAdUi' // Add more selectors if necessary
				);
				for (var i = 0; i < ads.length; i++) {
						ads[i].style.display = 'none';  // Hide the elements
				}

				// Remove elements from partner networks (example for MGID, adjust as needed)
				var mgid_widgets = document.querySelectorAll('[data-type="_mgwidget"]');
				mgid_widgets.forEach(widget => widget.remove());

				// Remove revive adserver
				var rv_widgets = document.querySelectorAll('[data-revive-zoneid]');
				rv_widgets.forEach(widget => widget.remove());

				// Remove elements with specific ids (customize as needed)
				var billboard_ad = document.getElementById('billboard_ad');
				if (billboard_ad) { billboard_ad.remove(); }
		"""
		driver.execute_script(js_code) # execute js code
	except Exception as e:
		 print(f"Error executing JavaScript: {e}")

#### Fungsi get_html()


In [13]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

def get_html(url, driver):
		try:
			driver.get(url)
			hide_ads(driver)
			html = driver.page_source
			return bs(html, "html5lib")
		
		except Exception as e:
			print(f"Error fetching URL: {e}")
			return None 

Fungsi get_html dengan parameter url digunakan untuk mengambil response atau isi html dari web. Untuk mengambil response tersebut dibutuhkan library request, dan juga BeautifulSoup untuk mendapatkan isi html.


#### Fungsi get_news()


In [14]:
def get_news(soup):
	container = soup.find("div", class_="flex flex-col gap-5")
	news_list = container.find_all("article", class_="flex-grow")
	return news_list

Fungsi get_news berfungsi untuk mengambil semua berita yang ada pada web, yang kemudian didapat kumpulan url berita yang ada pada halaman web.


#### Main Crawling


In [15]:
def main(nasional: int, internasional: int):
	options = Options()
	prefs = {
		"profile.managed_default_content_settings.images": 2,  # Disable images
		"profile.managed_default_content_settings.stylesheet": 2,  # Disable CSS
	}
	profile_path = "C:/Users/LAB SISTER/AppData/Local/Google/Chrome/User Data/Profile 7"
	options.add_experimental_option("prefs", prefs)
	options.add_argument(f"--user-data-dir={profile_path}")
	options.add_argument("--headless=new") # Run in headless mode
	driver = webdriver.Chrome(options=options)

	url = ["https://www.cnnindonesia.com/nasional/indeks/3/?page=", "https://www.cnnindonesia.com/internasional/indeks/6/?page="]
	# url = ["https://www.cnnindonesia.com/indeks/2/"]
	news = []

	nasional_count = 0
	internasional_count = 0

	try:
		while url:
			url_now = url.pop()
			count = 1 # - n
			halaman = int(get_html(url_now, driver).find("div", class_="flex gap-5 my-8 items-center justify-center undefined").find_all("a")[-2].text)

			for _ in range(halaman):
				page = f"{url_now}{count}" 
				soup = get_html(page, driver)
				news_list = get_news(soup)
				
				for item in tqdm(news_list, desc=f"Processing page {count}"):
					news_url = item.find('a')['href']
					
					try:
						result = scrape_news(get_html(news_url, driver))

						if result:
							if nasional_count >= nasional and internasional_count >= internasional:
								return news

							if result['kategori'] == 'nasional' and nasional_count < nasional:
								news.append(result)
								nasional_count += 1

							elif result['kategori'] == 'internasional' and internasional_count < internasional:
								news.append(result)
								internasional_count += 1
					except Exception as e:
						print(f"Error fetching news: {e}")
						continue
						
					
				count+=1
				if (nasional_count >= nasional and 'nasional' in url_now) or (internasional_count >= internasional and 'internasional' in url_now):
					break
	finally:
		driver.quit()

	return news

Menyiapkan link/base url web berita yang ingin dicrawling, terdapat beberapa fungsi yang dipanggil yang sudah dibuat sebelumnya untuk mengambil informasi atau berita pada halaman website. Dalam code tersebut terdapat beberapa tahapan seperti fungsi:

- get_html
- get_news
- scrape_news


#### Main


In [16]:
news = main(nasional=50, internasional=50)

Processing page 5: 100%|██████████| 10/10 [02:45<00:00, 16.51s/it]


Menjalankan program yang sudah dibuat dengan input berapa halaman yang ingin diambil.


### Convert Dataframe


In [17]:
df = pd.DataFrame.from_dict(news)
df
df.to_csv("data_100new.csv", index=False)

In [18]:
print(df['isi'][0])

Presiden Rusia Vladimir Putin menyebut akan tetap melanjutkan eksperimen rudal balistik dan hipersonik ke Ukraina.
Rusia menguji coba rudal hipersonik tersebut ke wilayah Ukraina, pada Kamis (21/11), menandai babak baru dalam konflik antara kedua negara. Penembakan rudal tersebut dilakukan setelah Gedung Putih mengizinkan Ukraina untuk menembakkan rudal jarak jauhnya ke Rusia.

Dalam sebuah pertemuan dengan pimpinan kementerian pertahanan Rusia, Putin mengklaim bahwa rudal tersebut tidak dapat dicegat oleh pertahanan udara. Ia juga mengatakan Rusia akan memulai produksi massal senjata barunya ini.
"Saat ini tidak ada cara untuk melawan rudal semacam itu, tidak ada cara untuk mencegatnya, di dunia. Dan saya akan menekankan sekali lagi: Kami akan terus menguji sistem terbaru," kata Putin, dikutip dari CNN.
Putin juga menyebut Rusia sedang mengembangkan beberapa sistem persenjataan serupa yang akan diuji coba lebih lanjut.


"Berdasarkan hasil uji coba, senjata-senjata ini juga akan mulai